# Using Jupyter Notebooks
:label:`sec_jupyter`


This section describes how to edit and run the code
in each section of this book
using the Jupyter Notebook. Make sure you have
installed Jupyter and downloaded the
code as described in
:ref:`chap_installation`.
If you want to know more about Jupyter see the excellent tutorial in
their [documentation](https://jupyter.readthedocs.io/en/latest/).


## Editing and Running the Code Locally

Suppose that the local path of the book's code is `xx/yy/d2l-en/`. Use the shell to change the directory to this path (`cd xx/yy/d2l-en`) and run the command `jupyter notebook`. If your browser does not do this automatically, open http://localhost:8888 and you will see the interface of Jupyter and all the folders containing the code of the book, as shown in :numref:`fig_jupyter00`.

![The folders containing the code of this book.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter00.png?raw=1)
:width:`600px`
:label:`fig_jupyter00`


You can access the notebook files by clicking on the folder displayed on the webpage.
They usually have the suffix ".ipynb".
For the sake of brevity, we create a temporary "test.ipynb" file.
The content displayed after you click it is
shown in :numref:`fig_jupyter01`.
This notebook includes a markdown cell and a code cell. The content in the markdown cell includes "This Is a Title" and "This is text.".
The code cell contains two lines of Python code.

![Markdown and code cells in the "text.ipynb" file.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter01.png?raw=1)
:width:`600px`
:label:`fig_jupyter01`


Double click on the markdown cell to enter edit mode.
Add a new text string "Hello world." at the end of the cell, as shown in :numref:`fig_jupyter02`.

![Edit the markdown cell.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter02.png?raw=1)
:width:`600px`
:label:`fig_jupyter02`


As demonstrated in :numref:`fig_jupyter03`,
click "Cell" $\rightarrow$ "Run Cells" in the menu bar to run the edited cell.

![Run the cell.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter03.png?raw=1)
:width:`600px`
:label:`fig_jupyter03`

After running, the markdown cell is shown in :numref:`fig_jupyter04`.

![The markdown cell after running.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter04.png?raw=1)
:width:`600px`
:label:`fig_jupyter04`


Next, click on the code cell. Multiply the elements by 2 after the last line of code, as shown in :numref:`fig_jupyter05`.

![Edit the code cell.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter05.png?raw=1)
:width:`600px`
:label:`fig_jupyter05`


You can also run the cell with a shortcut ("Ctrl + Enter" by default) and obtain the output result from :numref:`fig_jupyter06`.

![Run the code cell to obtain the output.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/jupyter06.png?raw=1)
:width:`600px`
:label:`fig_jupyter06`


When a notebook contains more cells, we can click "Kernel" $\rightarrow$ "Restart & Run All" in the menu bar to run all the cells in the entire notebook. By clicking "Help" $\rightarrow$ "Edit Keyboard Shortcuts" in the menu bar, you can edit the shortcuts according to your preferences.

## Advanced Options

Beyond local editing two things are quite important: editing the notebooks in the markdown format and running Jupyter remotely.
The latter matters when we want to run the code on a faster server.
The former matters since Jupyter's native ipynb format stores a lot of auxiliary data that is
irrelevant to the content,
mostly related to how and where the code is run.
This is confusing for Git, making
reviewing contributions very difficult.
Fortunately there is an alternative---native editing in the markdown format.

### Markdown Files in Jupyter

If you wish to contribute to the content of this book, you need to modify the
source file (md file, not ipynb file) on GitHub.
Using the notedown plugin we
can modify notebooks in the md format directly in Jupyter.


First, install the notedown plugin, run the Jupyter Notebook, and load the plugin:

```
pip install d2l-notedown  # You may need to uninstall the original notedown.
jupyter notebook --NotebookApp.contents_manager_class='notedown.NotedownContentsManager'
```

You may also turn on the notedown plugin by default whenever you run the Jupyter Notebook.
First, generate a Jupyter Notebook configuration file (if it has already been generated, you can skip this step).

```
jupyter notebook --generate-config
```

Then, add the following line to the end of the Jupyter Notebook configuration file (for Linux or macOS, usually in the path `~/.jupyter/jupyter_notebook_config.py`):

```
c.NotebookApp.contents_manager_class = 'notedown.NotedownContentsManager'
```

After that, you only need to run the `jupyter notebook` command to turn on the notedown plugin by default.

### Running Jupyter Notebooks on a Remote Server

Sometimes, you may want to run Jupyter notebooks on a remote server and access it through a browser on your local computer. If Linux or macOS is installed on your local machine (Windows can also support this function through third-party software such as PuTTY), you can use port forwarding:

```
ssh myserver -L 8888:localhost:8888
```

The above string `myserver` is the address of the remote server.
Then we can use http://localhost:8888 to access the remote server `myserver` that runs Jupyter notebooks. We will detail on how to run Jupyter notebooks on AWS instances
later in this appendix.

### Timing

We can use the `ExecuteTime` plugin to time the execution of each code cell in Jupyter notebooks.
Use the following commands to install the plugin:

```
pip install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
jupyter nbextension enable execute_time/ExecuteTime
```

## Summary

* Using the Jupyter Notebook tool, we can edit, run, and contribute to each section of the book.
* We can run Jupyter notebooks on remote servers using port forwarding.


## Exercises

1. Edit and run the code in this book with the Jupyter Notebook on your local machine.
1. Edit and run the code in this book with the Jupyter Notebook *remotely* via port forwarding.
1. Compare the running time of the operations $\mathbf{A}^\top \mathbf{B}$ and $\mathbf{A} \mathbf{B}$ for two square matrices in $\mathbb{R}^{1024 \times 1024}$. Which one is faster?


[Discussions](https://discuss.d2l.ai/t/421)


In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, concatenate, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Define paths
zip_file_path = "/content/dataset.zip"  # Path where the zip file will be uploaded
extracted_folder = "/content/extracted_dataset" # Path where the zip file will be extracted
dataset_path = os.path.join(extracted_folder, "Combined Dataset") # Path to the dataset
train_dir = os.path.join(dataset_path, "train")

# Define image size and batch size
image_size = (224, 224)
batch_size = 32

# Function to unzip the file
def unzip_file(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Check if the zip file exists
if os.path.exists(zip_file_path):
    print("Zip file found. Unzipping...")
    unzip_file(zip_file_path, extracted_folder)
    print("Zip file unzipped successfully.")
else:
    print("Zip file not found. Please upload the zip file to /content/dataset.zip")
    exit()

# Load the training dataset with data augmentation
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False,
    validation_split=0.2,
    subset="validation",
    seed=42
)

# Data augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

def augment_map(image, label):
    return data_augmentation(image), label

train_dataset = train_dataset.map(augment_map)

# Load pre-trained models without classification layers
densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=(image_size[0], image_size[1], 3))
mobilenet = MobileNet(include_top=False, weights='imagenet', input_shape=(image_size[0], image_size[1], 3))

# Freeze the pre-trained layers
for layer in densenet.layers:
    layer.trainable = False
for layer in mobilenet.layers:
    layer.trainable = False

# Create input layers
input_tensor = Input(shape=(image_size[0], image_size[1], 3))

# Get the output of the pre-trained models
densenet_output = densenet(input_tensor)
mobilenet_output = mobilenet(input_tensor)

# Global average pooling
densenet_pooled = GlobalAveragePooling2D()(densenet_output)
mobilenet_pooled = GlobalAveragePooling2D()(mobilenet_output)

# Feature fusion using concatenation
merged = concatenate([densenet_pooled, mobilenet_pooled])

# Add a dropout layer for regularization
dropout = Dropout(0.5)(merged)

# Add a classification layer
output = Dense(units=4, activation='softmax')(dropout)

# Create the final model
model = Model(inputs=input_tensor, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=[early_stopping])

# Save the model
model.save('fusion_model.h5')


In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, concatenate, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
# prompt: i want to combine  and train densenet,mobilenet using feature fusion into a single model which has both advantages such as densenet accuracy and architectuire mobilenet lightweight  on zip file.first i want to upload zip file and next unzip it and access the folders.the unzip folder contains combined dataset folder which again contain train ,test folders.i  just want to use  train folder to train .dont use test folder to validate.give me complete code for my project to train the single model with high accuracy.

import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, concatenate, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Define paths
zip_file_path = "/content/dataset.zip"  # Path where the zip file will be uploaded
extracted_folder = "/content/extracted_dataset" # Path where the zip file will be extracted
dataset_path = os.path.join(extracted_folder, "Combined Dataset") # Path to the dataset
train_dir = os.path.join(dataset_path, "train")

# Define image size and batch size
image_size = (224, 224)
batch_size = 32

# Function to unzip the file
def unzip_file(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Upload the zip file (if not already uploaded)
from google.colab import files
uploaded = files.upload()
if 'dataset.zip' in uploaded:
    print("Zip file uploaded successfully.")
    unzip_file(zip_file_path, extracted_folder)
    print("Zip file unzipped successfully.")
elif os.path.exists(zip_file_path):
    print("Zip file found. Unzipping...")
    unzip_file(zip_file_path, extracted_folder)
    print("Zip file unzipped successfully.")
else:
    print("Zip file not found. Please upload the zip file.")
    exit()


# Load the training dataset with data augmentation
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

# Data augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

def augment_map(image, label):
    return data_augmentation(image), label

train_dataset = train_dataset.map(augment_map)

# Load pre-trained models without classification layers
densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=(image_size[0], image_size[1], 3))
mobilenet = MobileNet(include_top=False, weights='imagenet', input_shape=(image_size[0], image_size[1], 3))

# Freeze the pre-trained layers
for layer in densenet.layers:
    layer.trainable = False
for layer in mobilenet.layers:
    layer.trainable = False

# Create input layers
input_tensor = Input(shape=(image_size[0], image_size[1], 3))

# Get the output of the pre-trained models
densenet_output = densenet(input_tensor)
mobilenet_output = mobilenet(input_tensor)

# Global average pooling
densenet_pooled = GlobalAveragePooling2D()(densenet_output)
mobilenet_pooled = GlobalAveragePooling2D()(mobilenet_output)

# Feature fusion using concatenation
merged = concatenate([densenet_pooled, mobilenet_pooled])

# Add a dropout layer for regularization
dropout = Dropout(0.5)(merged)

# Get the number of classes
num_classes = len(train_dataset.class_names)

# Add a classification layer
output = Dense(units=num_classes, activation='softmax')(dropout)

# Create the final model
model = Model(inputs=input_tensor, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(train_dataset, epochs=50, callbacks=[early_stopping])

# Save the model
model.save('fusion_model.h5')